In [1]:
from graph import *
from models import *
from metrics import *

import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

In [2]:
# load data from the spreadsheet which defines the structure of the workflow,
# as well as the parameters for data rates, efficiency, data reduction, and classifier performance
# ...these are taken from predictions for the Run-5 CMS
run5_system = dataframes_from_spreadsheet("cms_system_200.xlsx")

In [3]:
run5_system.detectors

,Category,Detector,Data (bytes),Sample Rate,Compression,Link Efficiency (J/bit),Op Efficiency (J/op),PU 200
0,Tracking,Inner Tracker,1440000,40000000,0,2.220000e-11,0,1.440
1,Tracking,Outer Tracker PS,720000,40000000,0,2.220000e-11,0,0.720
2,Tracking,Outer Tracker 2S,430000,40000000,0,2.220000e-11,0,0.430
3,Tracking,Track Finder TPG,10000,40000000,0,2.220000e-11,0,0.010
4,Timing,MIP Timing BTL,240000,40000000,0,2.220000e-11,0,0.240
5,Timing,MIP Timing ETL,440000,40000000,0,2.220000e-11,0,0.440
6,Calorimetry,ECAL Barrel,600000,40000000,0,2.220000e-11,0,0.600
7,Calorimetry,HCAL Barrel,240000,40000000,0,2.220000e-11,0,0.240
8,Calorimetry,HCAL HO,30000,40000000,0,2.220000e-11,0,0.030
9,Calorimetry,HCAL HF,60000,40000000,0,2.220000e-11,0,0.060


In [4]:
run5_system.triggers

,Name,Output,Data (bytes),Reduction Ratio,Classifier,Skill mean,Skill variance,Link Efficiency (J/bit),Op Efficiency (J/op),Compression
0,Tracking,Intermediate,0,1.0,Dummy,0,0,2.500000e-11,0.000,0
1,Timing,Intermediate,0,1.0,Dummy,0,0,2.500000e-11,0.000,0
2,Calorimetry,Intermediate,0,1.0,Dummy,0,0,2.500000e-11,0.000,0
3,Muon,Intermediate,0,1.0,Gaussian,0,0,2.500000e-11,0.000,0
4,Intermediate,Global,260000,53.3,L1T,0,0,2.500000e-11,0.003,0
5,Global,Disk,0,100.0,Gaussian,4,1,2.500000e-11,16.000,0
6,Disk,NaN,0,1.0,Dummy,0,0,2.500000e-11,0.000,0


In [5]:
#calculate the total reduction to keep this constant across experiments
overall_reduction = run5_system.triggers.iloc[4]["Reduction Ratio"] * run5_system.triggers.iloc[5]["Reduction Ratio"]

In [6]:
overall_reduction

5330.0

In [7]:
40e6 / overall_reduction

7504.6904315197

In [8]:
#import the data predicting wall time scaling by pileup
scaling = pd.read_excel("wall time scaling.xlsx", sheet_name="Data")
#fit a polynomial to this data for CPU and GPU runtimes
fit_poly = lambda x, k3, k2, k1: k3 * x ** 3 + k2 * x ** 2 + k1 * x
k, cv = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time"])
k_gpu, cv_gpu = curve_fit(fit_poly, scaling["Size"], scaling["Wall Time GPU"])

In [9]:
#define a dictionary with functions defining the scaling of trigger runtimes with incoming data
funcs = {"Global": lambda x: fit_poly(x, *k), "Intermediate": lambda x: x / 2.0e6}
funcs_gpu = {"Global": lambda x: fit_poly(x, *k_gpu), "Intermediate": lambda x: x / 2.0e6}

In [10]:
"""
Vary the accept rate of the level 1 trigger and inspect its impact on performance and resources required
"""
def init_system(system: System, functions, reduction: float, skill: float):
    t = system.triggers.copy()
    #intermediate reduction stage
    t.at[4, "Reduction Ratio"] = reduction
    #global reduction stage
    t.at[5, "Reduction Ratio"] = overall_reduction / reduction
    #change L1T skill
    t.at[4, "Skill mean"] = skill
    g = construct_graph(system.detectors, t, system.globals, functions)

    return g

In [11]:
reduction_to_ratio(0.0)

1.0

In [12]:
#current l1t accept / skill
ex = init_system(run5_system, funcs, 400, 3)

/Users/wilkie/code/system_flow/classifier.py:197: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  fit = lambda l: np.abs(self.egamma_rate - quad(lambda x: self.exp_dist(x, l) * interpolator(x), np.min(xs), np.max(xs))[0])


In [13]:
ex.nodes["Intermediate"]["reduction ratio"]

400.0

In [14]:
ex.nodes["Intermediate"]["classifier"].skill_boost

0.0

In [15]:
ratio_to_reduction(ex.nodes["Intermediate"]["reduction ratio"])

0.9975

In [16]:
ex.graph["op power"] / 1e6

44.959404950441446

In [17]:
ex.nodes["Intermediate"]["output rate"]

99663

In [18]:
ex.nodes["Intermediate"]["input rate"] / ex.nodes["Intermediate"]["output rate"]

401.3525480870534

In [19]:
ex.nodes["Intermediate"]["input rate"]

39999999

In [20]:
ex.nodes["Global"]["input rate"] / ex.nodes["Global"]["output rate"]

13.32571199358203

In [21]:
ex.nodes["Global"]["output rate"]

7479

In [22]:
ex.nodes["Global"]["input rate"]

99663

In [23]:
from copy import deepcopy

In [24]:
def vary_system(graph, reduction_ratio: float, skill: float):
    graph = deepcopy(graph)
    graph.nodes["Intermediate"]["reduction ratio"] = reduction_ratio
    graph.nodes["Intermediate"]["classifier"].skill_boost = skill

    graph = update_throughput(graph)

    power = graph.graph["op power"] + graph.graph["link power"]
    confusion = graph.graph["performance"]

    return power, confusion, graph

In [25]:
r = vary_system(ex, 100, 0.0)

In [26]:
r[0] / 1e6

178.8889799389007

In [27]:
#its predicted confusion matrix for the workflow
r[1]

array([[39951806,    18198],
       [   18173,    11818]])

In [28]:
s0 = vary_system(ex, 100, 0.0)

In [29]:
np.sum(get_passed(s0[1]))

29991

In [30]:
s0[2].nodes["Intermediate"]["classifier"].threshold

1.2856736459628448

In [31]:
s0[1]

array([[39951806,    18198],
       [   18173,    11818]])

In [32]:
s2 = vary_system(ex, 100, 2.0)

In [33]:
np.sum(get_passed(s2[1]))

30028

In [34]:
s2[1]

array([[39967854,     2114],
       [    2125,    27903]])

In [35]:
s2[2].nodes["Intermediate"]["classifier"].threshold

1.2966597811377931

In [36]:
s10 = vary_system(ex, 100, 10.0)

In [37]:
s10[2].nodes["Intermediate"]["classifier"].threshold

1.296616944245815

In [38]:
s10[1]

array([[39967854,     2114],
       [    2125,    27903]])

In [39]:
#its predicted confusion matrix for the workflow
contingency_to_error(r[1])

array([[9.99545334e-01, 6.06276652e-01],
       [4.54666238e-04, 3.93723348e-01]])

In [40]:
vary_system(ex, 400, 5)

(45003959.84973983,
 array([[39991987,      544],
        [     506,     6959]]),
 <networkx.classes.digraph.DiGraph at 0x16a350a00>)

In [41]:
vary_system(ex, 100, 5)

(179108879.64240992,
 array([[39967854,     2114],
        [    2125,    27903]]),
 <networkx.classes.digraph.DiGraph at 0x16a3fd690>)

In [42]:
#vary this accept rate from today's rate to the planned Run-5 
l1t_reductions = np.linspace(400, 53.3, 101)
l1t_skills = np.linspace(0, 1.0, 101)

In [43]:
res = [[vary_system(ex, r, s) for r in l1t_reductions] for s in l1t_skills]

In [44]:
def extract_metrics(results):
    all_confusion = np.array([r[1] for r in results])

    all_power = [r[0] / density_scale_model(r[2].graph["globals"]["Year"][0]) for r in results]
    all_power = np.array(all_power)

    all_recall = np.array([recall(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    all_f1 = np.array([f1_score(all_confusion[i,:,:]) for i in range(all_confusion.shape[0])])
    productivity = np.array([np.sum(get_passed(all_confusion[i,:,:])) for i in range(all_confusion.shape[0])])

    metrics = {"confusion": all_confusion,
               "power": all_power,
               "recall": all_recall,
               "f1 score": all_f1,
               "productivity": all_recall * productivity}

    return metrics

In [45]:
run5_metrics = [extract_metrics(r) for r in res]

In [46]:
f1 = np.stack([r["f1 score"] for r in run5_metrics])

In [47]:
power = np.stack([r["power"] for r in run5_metrics])

In [53]:
from scipy.ndimage import gaussian_filter

In [64]:
smoothed_f1 = gaussian_filter(f1, sigma=3)

In [72]:
fig = go.Figure(data =
    go.Contour(
        z=smoothed_f1,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         
    ),
    )
fig.update_layout(width = 800, 
                  height = 600,
                  xaxis_title = "L1T Reduction Ratio",
                  yaxis_title = "L1T Skill",
                  title = "Recall by L1T Skill & Reduction Ratio")
fig.update_xaxes(autorange="reversed")
fig.show()

In [68]:
fig = go.Figure(data =
    go.Contour(
        z=smoothed_f1,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800, 
                  height = 600,
                  title = "Recall by Trigger Skill & Reduction Ratio",
                  xaxis=dict(
                        title="Reduction Ratio",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the x-axis title
                    ),
                    yaxis=dict(
                        title="Skill",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the y-axis title
                    ),
                    font = dict(size=18,),)
fig.update_xaxes(autorange="reversed")
fig.show()

In [50]:
fig = go.Figure(data =
    go.Contour(
        z=power,
        x=l1t_reductions, # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800, 
                  height = 600,
                  title = "Power by Trigger Skill & Reduction Ratio",
                  xaxis=dict(
                        title="Reduction Ratio",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the x-axis title
                    ),
                    yaxis=dict(
                        title="Skill",
                        titlefont=dict(size=24, family='Arial, bold')  # Bold font for the y-axis title
                    ),
                    font = dict(size=18,),)
fig.update_xaxes(autorange="reversed")
fig.show()

In [51]:
#because its rejection is so much higher, there's more potential improvement gained by making L1T's skill higher 
#than simply passing more data to the HLT

In [73]:
fig = go.Figure(data =
    go.Contour(
        z = smoothed_f1,
        x=power[0,:], # horizontal axis
        y=l1t_skills, # vertical axis,
         contours_coloring='heatmap',
    ),
    )
fig.update_layout(width = 800,
                  height = 600,
                  title = "Performance Contours by Power and Skill",
                  xaxis_title = "Power",
                  yaxis_title = "L1T Skill")
fig.show()